# Tutorial: creación de ETLs con PySpark

In [8]:
import os 
from pyspark.sql import functions as f, SparkSession, types as t
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import udf, col, length, isnan, when, count, regexp_replace
from datetime import datetime

In [9]:
# Configuración servidor base de datos transaccional
# Recuerde usar Estudiante_i como usuario y la contraseña asigana en el excel de conexión a maquina virtual como contraseña
db_user = 'Estudiante_36_202314'
db_psswd = 'aabb1122'
source_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/ProyectoTransaccional'
dest_db_connection_string = 'jdbc:mysql://157.253.236.116:8080/Proyecto_G3_202314'
# Driver de conexion
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [ ]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

In [11]:
def obterner_dataframe_desde_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

def guardar_db(db_connection_string, df, tabla, db_user, db_psswd):
    df.select('*').write.format('jdbc') \
      .mode('append') \
      .option('url', db_connection_string) \
      .option('dbtable', tabla) \
      .option('user', db_user) \
      .option('password', db_psswd) \
      .option('driver', 'com.mysql.cj.jdbc.Driver') \
      .save()

In [ ]:
Tipos de Trafico (Esperados I, N y E)

In [28]:
sql_trafico =  '''
(SELECT ROW_NUMBER() OVER (ORDER BY trafico) AS idTrafico_DWH,
       v.trafico AS idTrafico_T,
       (CASE
           WHEN v.trafico = 'I' THEN 'Internacional'
           WHEN v.trafico = 'N' THEN 'Nacional'
           WHEN v.trafico = 'E' THEN 'Postal o Urgentes'
           ELSE 'Desconocido'
       END) AS nombreTrafico
FROM (
    SELECT DISTINCT trafico
    FROM ProyectoTransaccional.vuelos
) AS v) as Trafico
'''
Trafico = obtener_dataframe_de_bd(source_db_connection_string, sql_trafico, db_user, db_psswd)
Trafico.show()

+-------------+-----------+-----------------+
|idTrafico_DWH|idTrafico_T|    nombreTrafico|
+-------------+-----------+-----------------+
|            1|          E|Postal o Urgentes|
|            2|          I|    Internacional|
|            3|          N|         Nacional|
+-------------+-----------+-----------------+



In [ ]:
Transformaciones 


In [ ]:
Trafico.show().orderBy(col('idTrafico_DWH'))

In [32]:
Trafico = Trafico.withColumn('idTrafico_DWH',col('idTrafico_DWH').cast('int')).orderBy(col('idTrafico_DWH'))
Trafico.show()

+-------------+-----------+-----------------+
|idTrafico_DWH|idTrafico_T|    nombreTrafico|
+-------------+-----------+-----------------+
|            1|          E|Postal o Urgentes|
|            2|          I|    Internacional|
|            3|          N|         Nacional|
+-------------+-----------+-----------------+



In [33]:
# CARGUE
guardar_db(dest_db_connection_string,Trafico,'Proyecto_G3_202314.Trafico', db_user, db_psswd)